In [1]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import shutil
import time
!pip install pandas
import pandas as pd
!pip install unidecode
import unidecode

In [2]:
driver = webdriver.Chrome(
    executable_path="/Users/ericr/chromedriver_32/chromedriver"
    )

C:\Users\ericr\AppData\Local\Temp\ipykernel_11376\2497404269.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [2]:
years = list(range(1960, 2024))


In [5]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_totals.html"

for year in years:
    url = player_stats_url.format(year)
    
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(18)
    
    with open("../player_total/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(driver.page_source)

In [64]:
from bs4 import BeautifulSoup

dfs = []
for year in years:
    with open("../player_total/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    player_table = soup.find_all(id="div_totals_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs.append(player_df)

In [65]:
players_total = pd.concat(dfs)

In [66]:
players_total.head()


,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Gary Alcorn,C,23,DET,58,NaN,670,91,312,...,NaN,NaN,279,22,NaN,NaN,NaN,123,230,1960
1,2,Bob Anderegg,SF,22,NYK,33,NaN,373,55,143,...,NaN,NaN,69,29,NaN,NaN,NaN,32,133,1960
2,3,Paul Arizin*,SF,31,PHW,72,NaN,2618,593,1400,...,NaN,NaN,621,165,NaN,NaN,NaN,263,1606,1960
3,4,Dick Barnett,SG,23,SYR,57,NaN,1235,289,701,...,NaN,NaN,155,160,NaN,NaN,NaN,98,706,1960
4,5,Elgin Baylor*,SF,25,MNL,70,NaN,2873,755,1781,...,NaN,NaN,1150,243,NaN,NaN,NaN,234,2074,1960


In [67]:
players_total.to_csv("players_total.csv")

In [68]:
players = pd.read_csv("players_total.csv")


In [69]:
del players["Unnamed: 0"]

In [70]:
del players["Rk"]

In [71]:
players["Player"] = players["Player"].str.replace("*","", regex=False)

In [72]:
def single_team(df):
    if df.shape[0]==1:
        return df
    else:
        row = df[df["Tm"]=="TOT"]
        row["Tm"] = df.iloc[-1,:]["Tm"]
        return row

players = players.groupby(["Player", "Year"]).apply(single_team)

In [73]:
players

Player Pos Age   Tm   G  GS    MP   FG  \
Player         Year                                                         
A.C. Green     1986 6798       A.C. Green  PF  22  LAL  82   1  1542  209   
               1987 7197       A.C. Green  PF  23  LAL  79  72  2240  316   
               1988 7599       A.C. Green  PF  24  LAL  82  64  2636  322   
               1989 8048       A.C. Green  PF  25  LAL  82  82  2510  401   
               1990 8510       A.C. Green  PF  26  LAL  82  82  2709  385   
...                                   ...  ..  ..  ...  ..  ..   ...  ...   
Željko Rebrača 2002 14936  Željko Rebrača   C  29  DET  74   4  1179  189   
               2003 15436  Željko Rebrača   C  30  DET  30  12   488   80   
               2004 16017  Željko Rebrača   C  31  ATL  24   2   273   34   
               2005 16617  Željko Rebrača   C  32  LAC  58   2   928  133   
               2006 17211  Željko Rebrača   C  33  LAC  29   2   412   52   

                           FGA   FG%  ...  ORB  DRB  TRB  AST STL BLK  TOV  \
Player         Year                   ...                                    
A.C. Green     1986 6798   388  .539  ...  160  221  381   54  49  49   99   
               1987 7197   587  .538  ...  210  405  615   84  70  80  102   
               1988 7599   640  .503  ...  245  465  710   93  87  45  120   
               1989 8048   758  .529  ...  258  481  739  103  94  55  119   
               1990 8510   806  .478  ...  262  450  712   90  66  50  116   
...                        ...   ...  ...  ...  ...  ...  ...  ..  ..  ...   
Željko Rebrača 2002 14936  374  .505  ...   84  206  290   38  28  73   84   
               2003 15436  145  .552  ...   27   65   92    9   6  17   29   
               2004 16017   77  .442  ...   23   35   58    6   5  11   17   
               2005 16617  234  .568  ...   49  135  184   26  13  40   49   
               2006 17211   96  .542  ...   13   51   64    9   5  19   24   

                            PF   PTS  Year  
Player         Year                         
A.C. Green     1986 6798   229   521  1986  
               1987 7197   171   852  1987  
               1988 7599   204   937  1988  
               1989 8048   172  1088  1989  
               1990 8510   207  1061  1990  
...                        ...   ...   ...  
Željko Rebrača 2002 14936  192   513  2002  
               2003 15436   79   198  2003  
               2004 16017   52    91  2004  
               2005 16617  130   339  2005  
               2006 17211   59   135  2006  

[22351 rows x 30 columns]

In [74]:
players.index = players.index.droplevel()

In [75]:
players.index = players.index.droplevel()

In [85]:
players.head()
players.to_csv("players_total.csv")
players = pd.read_csv("players_total.csv")


In [87]:
from unidecode import unidecode
players['Player'] = players['Player'].apply(unidecode)


In [89]:
players.to_csv("players_total.csv")


In [79]:
import unidecode
dfs = []
special_str = players['Player']
for item in special_str:
    dfs = print(unidecode.unidecode(item))

A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.C. Green
A.J. Bramlett
A.J. English
A.J. English
A.J. Green
A.J. Guyton
A.J. Guyton
A.J. Guyton
A.J. Hammons
A.J. Lawson
A.J. Price
A.J. Price
A.J. Price
A.J. Price
A.J. Price
A.J. Price
A.J. Wynder
A.W. Holt
AJ Griffin
Aaron Brooks
Aaron Brooks
Aaron Brooks
Aaron Brooks
Aaron Brooks
Aaron Brooks
Aaron Brooks
Aaron Brooks
Aaron Brooks
Aaron Brooks
Aaron Gordon
Aaron Gordon
Aaron Gordon
Aaron Gordon
Aaron Gordon
Aaron Gordon
Aaron Gordon
Aaron Gordon
Aaron Gordon
Aaron Gray
Aaron Gray
Aaron Gray
Aaron Gray
Aaron Gray
Aaron Gray
Aaron Gray
Aaron Harrison
Aaron Harrison
Aaron Harrison
Aaron Henry
Aaron Holiday
Aaron Holiday
Aaron Holiday
Aaron Holiday
Aaron Holiday
Aaron Jackson
Aaron James
Aaron James
Aaron James
Aaron James
Aaron James
Aaron McKie
Aaron McKie
Aaron McKie
Aaron McKie
Aaron McKie
Aaron McKie
Aaron McKie
Aaro

In [81]:
dfs

In [37]:
players['Player'] = unidecode.unidecode(item)

In [62]:
players_total.head(400)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Gary Alcorn,C,23,DET,58,NaN,670,91,312,...,NaN,NaN,279,22,NaN,NaN,NaN,123,230,1960
1,2,Bob Anderegg,SF,22,NYK,33,NaN,373,55,143,...,NaN,NaN,69,29,NaN,NaN,NaN,32,133,1960
2,3,Paul Arizin,SF,31,PHW,72,NaN,2618,593,1400,...,NaN,NaN,621,165,NaN,NaN,NaN,263,1606,1960
3,4,Dick Barnett,SG,23,SYR,57,NaN,1235,289,701,...,NaN,NaN,155,160,NaN,NaN,NaN,98,706,1960
4,5,Elgin Baylor,SF,25,MNL,70,NaN,2873,755,1781,...,NaN,NaN,1150,243,NaN,NaN,NaN,234,2074,1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,21,Bob Cousy,PG,34,BOS,76,NaN,1975,392,988,...,NaN,NaN,193,515,NaN,NaN,NaN,175,1003,1963
23,22,Johnny Cox,SG,26,CHZ,73,NaN,1685,239,568,...,NaN,NaN,280,142,NaN,NaN,NaN,149,573,1963
24,23,Dave DeBusschere,SF,22,DET,80,NaN,2352,406,944,...,NaN,NaN,694,207,NaN,NaN,NaN,247,1018,1963
25,24,Terry Dischinger,SF,22,CHZ,57,NaN,2294,525,1026,...,NaN,NaN,458,175,NaN,NaN,NaN,188,1452,1963


In [63]:
players_total.to_csv("players_total.csv")